In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf

train_data = np.load('/content/drive/MyDrive/HW12/train_data.npy')
train_label = np.load('/content/drive/MyDrive/HW12/train_label.npy')
test_data = np.load('/content/drive/MyDrive/HW12/test_data.npy')

In [3]:
# Shuffle the training data
n_train = tf.shape(train_data)[0]
n_test = tf.shape(test_data)[0]

zip_train = list(zip(train_data, train_label))
np.random.shuffle(zip_train)
train_data, train_label = zip(*zip_train)

train_data = tf.reshape(train_data, (5000, 28, 28, 1))

In [5]:
print("Number of training set: "+ str(n_train))
print("Number of testing set: "+ str(n_test))

Number of training set: tf.Tensor(5000, shape=(), dtype=int32)
Number of testing set: tf.Tensor(100000, shape=(), dtype=int32)


In [6]:
# Define optimizers for feature extractor, classifier and domain discriminator
extractor_optimizer = tf.keras.optimizers.Adam(1e-4)
c_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

In [7]:
# Define feature extractor
extractor = tf.keras.Sequential([
                                 tf.keras.Input(shape=(28, 28, 1)),
                                 tf.keras.layers.Conv2D(64, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(128, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(256, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Conv2D(512, 3, strides=1, padding='same'),
                                 tf.keras.layers.BatchNormalization(),
                                 tf.keras.layers.LeakyReLU(),
                                 tf.keras.layers.MaxPool2D(2, 2, padding='same'),

                                 tf.keras.layers.Flatten(),

                                 tf.keras.layers.Dense(512)
                                 
])

In [8]:
# Define domain classifier
domain_classifier = tf.keras.Sequential([
                                     tf.keras.Input(shape=(512, )),
                                     tf.keras.layers.Dense(128, activation='relu'),
                                     tf.keras.layers.Dense(64, activation='relu'),
                                     tf.keras.layers.Dense(1)
])

In [9]:
# Define label classifier
label_classifier = tf.keras.Sequential([
                                        tf.keras.Input(shape=(512, )),
                                        tf.keras.layers.Dense(128, activation='relu'),
                                        tf.keras.layers.Dense(64, activation='relu'),
                                        tf.keras.layers.Dense(10, activation='softmax')
])

In [10]:
# Define losses for feature extractor
def extractor_loss(source_label, output_feature):
  true_label = source_label[:5000, :]

  source_feature = output_feature[:5000]
  target_feature = output_feature[5000:]

  category_cross_entropy = tf.keras.losses.CategoricalCrossentropy()
  source_label_output = label_classifier(source_feature)
  source_label_loss = category_cross_entropy(true_label, source_label_output)

  binary_cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  domain_output = domain_classifier(output_feature)
  domain_true = tf.concat((tf.zeros(5000, ), tf.ones(5000, )), axis=0)
  domain_loss = binary_cross_entropy(domain_true, domain_output) 

  lamb = 0.1

  return source_label_loss - lamb * domain_loss

In [11]:
extractor.compile(optimizer=extractor_optimizer, loss=extractor_loss)
domain_classifier.compile(optimizer=d_optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
label_classifier.compile(optimizer=c_optimizer, loss=tf.keras.losses.CategoricalCrossentropy())

In [ ]:
epochs = 50
source_data = train_data
source_label_for_c = tf.cast(train_label, tf.int8)
source_label_for_e = tf.concat((train_label, tf.zeros_like(train_label)), axis=0)
source_label_for_e = tf.cast(source_label_for_e, tf.int8)
for i in range(epochs):
  print("Current epoch is:"+str(i))
  for j in range(20):

    target_data = test_data[5000*j:5000*(j+1), :, :, :]
    target_data = tf.reshape(target_data, (5000, 28, 28, 1))

    x = tf.concat((source_data, target_data), axis=0)
    x = tf.reshape(x, (10000, 28, 28, 1))
    
    d_target = tf.concat((tf.zeros(5000, ), tf.ones(5000, )), axis=0)

    feature_output = extractor(x, training=True)
    source_feature = feature_output[:5000]
    target_feature = feature_output[5000:]
    # Train the domain classifier
    domain_classifier.fit(feature_output, d_target, batch_size=64, epochs=5, verbose=False)

    # Train the extractor and label_classifier
    label_classifier.fit(source_feature, source_label_for_c, batch_size=64, epochs=1, verbose=False)
    extractor.fit(x, source_label_for_e, batch_size=10000, epochs=1, shuffle=False, verbose=False)
    

In [26]:
pred_source_label = label_classifier(extractor(source_data))
source_error = 0
for i in range(5000):
  y_true = tf.where(train_label[i] == max(train_label[i]))
  y_pred = tf.where(pred_source_label[i] == max(pred_source_label[i]))
  
  if y_true != y_pred:
    source_error += 1

In [27]:
print(source_error)

49


In [34]:
pred_target_label = label_classifier(extractor(test_data[:10]))
results = []
for i in range(10):
  y_pred = tf.where(pred_target_label[i] == max(pred_target_label[i]))
  results.append(int(y_pred.numpy()))

print(results)

[0, 2, 6, 6, 2, 4, 2, 2, 5, 5]
